Importing important function from libraries

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.utils.np_utils import to_categorical
from keras.utils import np_utils
import matplotlib.pyplot as plt
import pandas as pd
import math

Training Data with 42000 images of 28*28 = 784 pixels each

In [6]:
train = pd.read_csv("/content/drive/MyDrive/Digit_Recognizer/train.csv")
print(train.shape)

(42000, 785)


Test Data with 28000 images of 28*28=784 pixels each

In [7]:
test = pd.read_csv("/content/drive/MyDrive/Digit_Recognizer/test.csv")
print(test.shape)

(28000, 784)


x_train is the input training data

y_train is the label/output of each input training image

x_test is the input test data

In [8]:
x_train = train.iloc[:, 1:785].values
y_train = train.iloc[:, 0].values
x_test = test.iloc[:, 0:784].values
print(x_train.shape)
print(y_train.shape)

(42000, 784)
(42000,)


Dividing the images by 255 to normalize them

In [9]:
seed = 5
np.random.seed(seed)
x_train = x_train/255.0
x_test = x_test/255.0
print(x_train.shape)

(42000, 784)


We want images in a 28*28 size array

In [10]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
print(x_train.shape)

(42000, 28, 28, 1)


Defining some constants

In [11]:
num_classes = 10          #We can have 10 different ouptuts
batch_size = 64           
epochs = 40               
input_shape = (28,28,1)   #Size of input image

Categorizing all outputs in 10 different possibilities

In [12]:
y_train = to_categorical(y_train, num_classes)
print(y_train.shape)

(42000, 10)


In [13]:
from sklearn.model_selection import train_test_split
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size = 0.2, random_state=seed)

**MODEL**

In [14]:
model = Sequential()
model.add(Conv2D(32, kernel_size = (3,3), input_shape= input_shape, activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3,3), activation = 'relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(512,activation = 'relu'))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(num_classes, activation = 'sigmoid'))

In [15]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [16]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0

In [17]:
model.fit(x_train, y_train, epochs = epochs, verbose = 1)

Epoch 1/40
1050/1050 [==============================] - 47s 3ms/step - loss: 0.4117 - accuracy: 0.8643
Epoch 2/40
1050/1050 [==============================] - 3s 3ms/step - loss: 0.0727 - accuracy: 0.9778
Epoch 3/40
1050/1050 [==============================] - 3s 3ms/step - loss: 0.0552 - accuracy: 0.9829
Epoch 4/40
1050/1050 [==============================] - 3s 3ms/step - loss: 0.0384 - accuracy: 0.9882
Epoch 5/40
1050/1050 [==============================] - 3s 3ms/step - loss: 0.0350 - accuracy: 0.9891
Epoch 6/40
1050/1050 [==============================] - 3s 3ms/step - loss: 0.0318 - accuracy: 0.9904
Epoch 7/40
1050/1050 [==============================] - 3s 3ms/step - loss: 0.0275 - accuracy: 0.9914
Epoch 8/40
1050/1050 [==============================] - 3s 3ms/step - loss: 0.0305 - accuracy: 0.9900
Epoch 9/40
1050/1050 [==============================] - 3s 3ms/step - loss: 0.0207 - accuracy: 0.9929
Epoch 10/40
1050/1050 [==============================] - 3s 3ms/step - loss: 0.02

In [18]:
loss , accuracy = model.evaluate(x_validation, y_validation, verbose = 0)
print("Loss : ",loss, "Accuracy : ", accuracy)

Loss :  0.0714588537812233 Accuracy :  0.9885714054107666


So, we got an accuracy of 98.88 percent and a loss of 0.7 % which is alright

In [19]:
predicted_classes = model.predict_classes(x_test)

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [20]:
submissions=pd.DataFrame({"ImageId": list(range(1,len(predicted_classes)+1)), "Label": predicted_classes})

In [21]:
submissions.to_csv("submission.csv", index = False, header = True)

In [22]:
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>